
# Open-Meteo API — Exercises (Solved)
**Python + Requests + JSON + Pandas + Matplotlib**  
This Colab notebook contains a guided, solved set of exercises to practice HTTP requests to public APIs (Open-Meteo), JSON handling, and basic data analysis/visualization with Pandas and Matplotlib.

> Tip: Run each cell in order. You can modify parameters (latitude/longitude, variables, etc.) to explore different locations and datasets.


In [ ]:

# == Setup ==
# In Google Colab these are usually pre-installed, but we include them just in case.
# You can uncomment the following lines if needed.
# !pip install requests pandas matplotlib --quiet

import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

print("Libraries loaded.")


## 0) Base example: current weather (from class)

In [ ]:

# 0) Base example: current weather
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 37.98,        # Torrevieja
    "longitude": -0.68,       # Torrevieja
    "current_weather": True   # Only current weather
}

response = requests.get(url, params=params)
data = response.json()

# Pretty print JSON
print(json.dumps(data, indent=4, ensure_ascii=False))

# Simple outputs
print("Temperatura actual:", data["current_weather"]["temperature"], "°C")
print("Velocidad del viento:", data["current_weather"]["windspeed"], "km/h")



## 🟢 Level 1 — Basic API understanding
### 1) Change the geographic location
**Task:** Query another city by changing latitude/longitude.  
**Solution below:** Example with Madrid (lat=40.4168, lon=-3.7038).


In [ ]:

url = "https://api.open-meteo.com/v1/forecast"
params = {"latitude": 40.4168, "longitude": -3.7038, "current_weather": True}  # Madrid

response = requests.get(url, params=params)
data_madrid = response.json()

print("== Madrid current weather ==")
print("Temperature:", data_madrid["current_weather"]["temperature"], "°C")
print("Wind speed:", data_madrid["current_weather"]["windspeed"], "km/h")



### 2) Explore documentation & add parameters
**Task:** Use additional parameters like `hourly`, `daily`, `forecast_days`.
**Solution:** Request hourly temperature and humidity for the current day.


In [ ]:

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 37.98,
    "longitude": -0.68,
    "hourly": ["temperature_2m", "relative_humidity_2m"],
    "forecast_days": 1
}

response = requests.get(url, params=params)
data_hourly = response.json()
print("Keys at root level:", list(data_hourly.keys()))
print("Keys under 'hourly':", list(data_hourly["hourly"].keys())[:10])
print("First 5 timestamps:", data_hourly["hourly"]["time"][:5])



## 🟡 Level 2 — Hourly/Daily data
### 3) Download a time series of temperature and humidity (hourly)
**Task:** Print the first 10 rows in a tidy way using Pandas.


In [ ]:

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 37.98,
    "longitude": -0.68,
    "hourly": ["temperature_2m", "relative_humidity_2m"],
    "forecast_days": 1
}

response = requests.get(url, params=params)
data_ts = response.json()

hourly = data_ts["hourly"]
df_hourly = pd.DataFrame(hourly)
print(df_hourly.head(10))



### 4) Visualize the hourly temperature for the day
**Task:** Plot `temperature_2m` vs. `time`.


In [ ]:

df_plot = df_hourly.copy()
df_plot["time"] = pd.to_datetime(df_plot["time"])

plt.figure()
plt.plot(df_plot["time"], df_plot["temperature_2m"])
plt.title("Hourly Temperature — Today")
plt.xlabel("Time")
plt.ylabel("Temperature (°C)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



### 5) Compute basic statistics
**Task:** Min, max, mean, and range (max - min) of the hourly temperature.


In [ ]:

t_min = df_hourly["temperature_2m"].min()
t_max = df_hourly["temperature_2m"].max()
t_mean = df_hourly["temperature_2m"].mean()
t_range = t_max - t_min

print(f"Min: {t_min:.2f} °C, Max: {t_max:.2f} °C, Mean: {t_mean:.2f} °C, Range: {t_range:.2f} °C")



## 🔵 Level 3 — Daily data & files
### 6) Download daily data for 7 days
**Task:** Use `daily = ["temperature_2m_max", "temperature_2m_min"]` and `forecast_days=7`.


In [ ]:

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 37.98,
    "longitude": -0.68,
    "daily": ["temperature_2m_max", "temperature_2m_min"],
    "forecast_days": 7,
    "timezone": "auto"
}

response = requests.get(url, params=params)
data_daily = response.json()

daily = data_daily["daily"]
df_daily = pd.DataFrame(daily)
print(df_daily)



### 7) Save result to JSON file
**Task:** Save the full API response to a local JSON file: `prevision.json`.


In [ ]:

with open("prevision.json", "w", encoding="utf-8") as f:
    json.dump(data_daily, f, ensure_ascii=False, indent=4)
print("Saved file: prevision.json")



### 8) Load JSON into a Pandas DataFrame and plot
**Task:** Extract `time`, `temperature_2m_max`, `temperature_2m_min` and plot.


In [ ]:

# If you want to ensure loading from file:
# with open("prevision.json", "r", encoding="utf-8") as f:
#     data_daily = json.load(f)

daily = data_daily["daily"]
df_daily = pd.DataFrame({
    "time": pd.to_datetime(daily["time"]),
    "tmax": daily["temperature_2m_max"],
    "tmin": daily["temperature_2m_min"]
})

print(df_daily)

plt.figure()
plt.plot(df_daily["time"], df_daily["tmax"], label="Tmax")
plt.plot(df_daily["time"], df_daily["tmin"], label="Tmin")
plt.title("Daily Max/Min Temperature — Next 7 Days")
plt.xlabel("Date")
plt.ylabel("Temperature (°C)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



## 🔴 Level 4 — Creative extensions
### 9) Compare two locations (Torrevieja vs Madrid) — daily max temps


In [ ]:

def get_daily_tmax(latitude, longitude, days=7):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": ["temperature_2m_max", "temperature_2m_min"],
        "forecast_days": days,
        "timezone": "auto"
    }
    resp = requests.get(url, params=params)
    d = resp.json()
    daily = d["daily"]
    return pd.DataFrame({
        "time": pd.to_datetime(daily["time"]),
        "tmax": daily["temperature_2m_max"],
        "tmin": daily["temperature_2m_min"]
    })

df_torrevieja = get_daily_tmax(37.98, -0.68, days=7)
df_madrid = get_daily_tmax(40.4168, -3.7038, days=7)

plt.figure()
plt.plot(df_torrevieja["time"], df_torrevieja["tmax"], label="Torrevieja Tmax")
plt.plot(df_madrid["time"], df_madrid["tmax"], label="Madrid Tmax")
plt.title("Daily Tmax — Torrevieja vs Madrid")
plt.xlabel("Date")
plt.ylabel("Temperature (°C)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



### 10) Analyze wind speed (hourly) for one day


In [ ]:

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 37.98,
    "longitude": -0.68,
    "hourly": ["windspeed_10m"],
    "forecast_days": 1,
    "timezone": "auto"
}

response = requests.get(url, params=params)
data_wind = response.json()

df_wind = pd.DataFrame(data_wind["hourly"])
df_wind["time"] = pd.to_datetime(df_wind["time"])

plt.figure()
plt.plot(df_wind["time"], df_wind["windspeed_10m"])
plt.title("Hourly Wind Speed — Today")
plt.xlabel("Time")
plt.ylabel("Wind speed (km/h)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



### 11) Compute a simple thermal index for the day
\N{BLACK SMALL SQUARE} **Index = (Tmax + Tmin) / 2** from daily data.


In [ ]:

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 37.98,
    "longitude": -0.68,
    "daily": ["temperature_2m_max", "temperature_2m_min"],
    "forecast_days": 1,
    "timezone": "auto"
}

response = requests.get(url, params=params)
data_one_day = response.json()
daily = data_one_day["daily"]

tmax = daily["temperature_2m_max"][0]
tmin = daily["temperature_2m_min"][0]
index = (tmax + tmin) / 2.0

print(f"Thermal Index ( (Tmax + Tmin)/2 ): {index:.2f} °C")



---

## ✅ Wrap-up
You practiced:
- Building HTTP queries with **`requests`**
- Parsing **JSON** and pretty-printing with `json.dumps(...)`
- Converting nested JSON to **Pandas DataFrames**
- Plotting with **Matplotlib** (single-plot charts)

> Try extending these exercises with more variables (precipitation, cloud cover, etc.) or by exporting results to CSV/Excel.
